## <u>Health Analytics Mini Case Study</u>

### An urgent request from the General Manager of Analytics at Health Co requesting our assistance with their analysis of the health.user_logs dataset.

### Business questions: 
1. How many unique users exist in the logs dataset?
~~~sql
  SELECT COUNT(DISTINCT id) 
  FROM health.user_logs

![image info](./week1_screenshots/s19.png)

2. How many total measurements do we have per user on average?
~~~sql 
DROP TABLE IF EXISTS user_measure_count;
CREATE TEMP TABLE user_measure_count AS (
SELECT
    id,
    COUNT(*) AS measure_count,
    COUNT(DISTINCT measure) as unique_measures
  FROM health.user_logs
  GROUP BY id); 

SELECT
  ROUND(AVG(measure_count),2) AS avg_measurements
FROM user_measure_count;

![image info](./week1_screenshots/s20.png)

3. What about the median number of measurements per user?
~~~sql
SELECT 
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY measure_count) AS median_value
FROM user_measure_count;

![image info](./week1_screenshots/s21.png)

4. How many users have 3 or more measurements?
~~~sql
SELECT 
  COUNT(*)
FROM user_measure_count
WHERE measure_count >= 3

![image info](./week1_screenshots/s22.png)

5. How many users have 1,000 or more measurements?
~~~sql
SELECT 
  COUNT(*)
FROM user_measure_count
WHERE measure_count >= 1000

![image info](./week1_screenshots/s23.png)

6. Looking at the logs data - what is the number and percentage of the active user base who:
   - Have logged blood glucose measurements?

~~~sql
SELECT measure,
  COUNT(DISTINCT id) AS unique_blood_glucose_user,
ROUND(100 * COUNT(DISTINCT id) / SUM(COUNT (DISTINCT ID)) OVER(),2) AS percent 
FROM health.user_logs
GROUP BY measure


![image info](./week1_screenshots/s24.png)

   - Have at least 2 types of measurements?
~~~sql
SELECT
  COUNT(*), ROUND(100 * COUNT(*) / SUM(unique_measures),2) AS percent
FROM user_measure_count
WHERE unique_measures >= 2;


![image info](./week1_screenshots/s25.png)

   - Have all 3 measures - blood glucose, weight and blood pressure?

   ~~~sql
   WITH all_measures AS (
SELECT *
FROM user_measure_count
WHERE unique_measures = 3)

SELECT
  COUNT(DISTINCT m.id) AS unique_user,
  100 * ROUND(COUNT(DISTINCT m.id)::numeric / COUNT(DISTINCT u.id),2) AS unique_user_percentage
FROM user_measure_count AS u
LEFT JOIN all_measures AS m
  ON u.id = m.id;

![image info](./week1_screenshots/s26.png)

7. For users that have blood pressure measurements:
   - What is the median systolic/diastolic blood pressure values?

~~~sql 
WITH t1 AS (
SELECT id, 
  measure, systolic, diastolic
  FROM 
  health.user_logs
  WHERE measure = 'blood_pressure'
)
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY systolic) AS median_syst, 
PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY diastolic) AS median_diastol
FROM t1


![image info](./week1_screenshots/s27.png)